### Setup

In [1]:
import os, sys
PWD = os.getenv('PWD')
print(PWD)

/protwis/sites/protwis


In [2]:
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "protwis.settings")
import django
django.setup()

/env/lib/python3.4/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


### Imports

In [3]:
from itertools import combinations
from tqdm import tqdm

from signprot.interactions import *
from protein.models import Protein, ProteinSegment
from signprot.models import SignprotComplex
from seqsign.sequence_signature import SequenceSignature

### Defining the protein sets

In [4]:
sets = {
    'Gi/o': [],
    'Gq/11': [],
    'Gs': [],
    'G12/13': [],
}

set_combinations = list(combinations(sets.keys(), 2))
segments = list(ProteinSegment.objects.filter(proteinfamily='GPCR'))

In [5]:
set_combinations

[('Gi/o', 'Gq/11'),
 ('Gi/o', 'G12/13'),
 ('Gi/o', 'Gs'),
 ('Gq/11', 'G12/13'),
 ('Gq/11', 'Gs'),
 ('G12/13', 'Gs')]

In [6]:
for gprot in sets.keys():
    structure_ids = list(
        SignprotComplex.objects.filter(
            protein__family__name=gprot
        ).values_list(
            'structure',
            flat=True
        ))
    
    sets[gprot] = Protein.objects.filter(proteinconformation__structure__in=structure_ids)
    

sets

{'G12/13': <QuerySet []>,
 'Gi/o': <QuerySet [<Protein: 6n4b>, <Protein: 6d9h>, <Protein: 6g79>, <Protein: 6ddf>, <Protein: 6dde>, <Protein: 6cmo>]>,
 'Gq/11': <QuerySet []>,
 'Gs': <QuerySet [<Protein: 5vai>, <Protein: 6b3j>, <Protein: 3sn6>, <Protein: 6gdg>, <Protein: 5uz7>, <Protein: 6niy>]>}

In [7]:
signatures = []
for combination in set_combinations:
    pos_set = sets[combination[0]]
    neg_set = sets[combination[1]]
    
    if pos_set and neg_set:
        signature = SequenceSignature()
        signature.setup_alignments(segments, pos_set, neg_set)
        signature.calculate_signature()


        signatures.append({
            'pos_set': {
                'group': combination[0],
                'objects': pos_set,
            },
            'neg_set': {
                'group': combination[1],
                'objects': pos_set,
            },
            'signature': signature,

        })


In [8]:
signatures

[{'neg_set': {'group': 'Gs',
   'objects': <QuerySet [<Protein: 6n4b>, <Protein: 6d9h>, <Protein: 6g79>, <Protein: 6ddf>, <Protein: 6dde>, <Protein: 6cmo>]>},
  'pos_set': {'group': 'Gi/o',
   'objects': <QuerySet [<Protein: 6n4b>, <Protein: 6d9h>, <Protein: 6g79>, <Protein: 6ddf>, <Protein: 6dde>, <Protein: 6cmo>]>},
  'signature': <seqsign.sequence_signature.SequenceSignature at 0x7fe2eae44d30>}]

In [18]:
for signature_dict in signatures:
    signature = signature_dict['signature']

    sig_data = signature.prepare_display_data()
    gn = get_generic_numbers(sig_data)
    gn_flat = list(chain.from_iterable(gn))
    
    signature_dict['consensus'] = []
    cons = get_signature_consensus(sig_data, gn_flat)
    for entry in cons:
        feature_code = entry['code']
        conservation = entry['cons']
        if feature_code != '-' and conservation == 10:
            signature_dict['consensus'].append(entry)

In [19]:
signatures

[{'consensus': [{'code': 'HY',
    'cons': 10,
    'feature': 'Hydrophobic',
    'gn': '1.35x35',
    'key': 114,
    'length': 'any',
    'score': 100},
   {'code': 'HA',
    'cons': 10,
    'feature': 'Hydrophobic aliphatic',
    'gn': '1.36x36',
    'key': 115,
    'length': 'any',
    'score': 100},
   {'code': 'HY',
    'cons': 10,
    'feature': 'Hydrophobic',
    'gn': '1.38x38',
    'key': 117,
    'length': 'any',
    'score': 100},
   {'code': 'HA',
    'cons': 10,
    'feature': 'Hydrophobic aliphatic',
    'gn': '1.42x42',
    'key': 121,
    'length': '2-3',
    'score': 100},
   {'code': 'HY',
    'cons': 10,
    'feature': 'Hydrophobic',
    'gn': '1.45x45',
    'key': 124,
    'length': 'any',
    'score': 100},
   {'code': 'HY',
    'cons': 10,
    'feature': 'Hydrophobic',
    'gn': '1.48x48',
    'key': 127,
    'length': 'any',
    'score': 100},
   {'code': 'N',
    'cons': 10,
    'feature': 'Hydrogen bonding [N]',
    'gn': '1.50x50',
    'key': 129,
    'length'